In [21]:
import numpy as np
from pathlib import Path
import re
from tqdm import tqdm
from skimage import io
from skimage.morphology import skeletonize_3d, binary_dilation, binary_closing
from scipy.ndimage import distance_transform_edt
import tifffile as tif
from scipy.ndimage import binary_fill_holes 
import cc3d
from scipy.io import loadmat, savemat
import skan
import sknw
import networkx as nx
import pickle
#from pystackreg import StackReg
#from plantcv import plantcv as pcv
#from VascGraph.Skeletonize import Skeleton

#from VascGraph.GraphIO import ReadStackMat
#from VascGraph.GraphLab import StackPlot
#from VascGraph.Tools.VisTools import visG
#from VascGraph.Tools.CalcTools import fixG
#from VascGraph.GraphIO import WritePajek

In [22]:
#!pip install pystackreg

In [1]:
def remove_small_comps_3d(image, thresh = 1000):
    """
    

    Parameters
    ----------
    image : binary np array with uint8 elements
        3d numpy matrix, connected components will be removed form this image
    thresh : int64
        smallest connected components to keep

    Returns
    -------
    np.array with uint8 elements, binary
        binary image with connected components below the threshold removed.

    """
    img_lab, N = cc3d.connected_components(image,return_N=True)
    unique, counts = np.unique(img_lab, return_counts=True)
    unique_keep = unique[counts>thresh]
    unique_keep = np.delete(unique_keep,[0])
    img_filt = np.zeros(img_lab.shape).astype('int8')
    img_filt[np.isin(img_lab,unique_keep)] = 1
    return img_filt.astype('uint8')   

def fill_holes(img):
    #res = np.zeros(img.shape)
    for i in np.unique(img)[::-1]:
        _tmp = (img==i)*1.0
        _tmp = _tmp.astype('int8')
        _tmp = remove_small_comps_3d(_tmp)
        img[_tmp==1] = i
    res = img.astype('int8')
    return res

In [24]:
directory = Path('james_preds')
files  = directory.glob('*_seg_warped.tif')
files = sorted([x.as_posix() for x in files])

In [25]:
files[0]

'james_preds/XYZres006_0001_seg_warped.tif'

In [26]:
min_prob = 0.75
max_var = 0.1
for file in tqdm(files):
    seg = io.imread(file)
    seg = seg.astype('int8')
    seg = (seg==1)*1
    seg = fill_holes(seg)
    #np.save(re.sub('mean','seg',file),seg)
    savemat(re.sub('_seg_warped.tif','_seg_warped.mat',file),{'FinalImage':fill_holes(binary_dilation(binary_dilation(seg)))})
    tif.imwrite(re.sub('_seg_warped.tif','_seg_warped_filled.tif',file),seg)

100%|██████████| 94/94 [29:48<00:00, 19.02s/it]


In [7]:
np.max(seg)

NameError: name 'seg' is not defined

In [27]:
files_seg_0001 = directory.glob('*_0001_seg_warped.tif')
files_seg_0001 = sorted([x.as_posix() for x in files_seg_0001])
len(files_seg_0001)

47

In [20]:
for file_0001 in tqdm(files_seg_0001):
    file = file_0001
    skel_file = re.sub('_seg_warped.tif','_skel_warped.mat',file)
    seg = io.imread(file)
    skel = loadmat(skel_file)
    skel = loadmat(skel_file)['FilteredImage']
    skel = skeletonize_3d(skel)
    skel = remove_small_comps_3d(skel,10)
    dst_tsfm = distance_transform_edt(seg)*0.99436
    dst_tsfm[dst_tsfm==0]=0.00001
    skel_dst = skel*dst_tsfm

    
    np.save(re.sub('seg','dst_skel',file),skel_dst)
    io.imsave(re.sub('_seg_warped.tif','_dst_skel_warped.tif',file),skel_dst)
    graph = sknw.build_sknw(skel_dst, multi=False)
    #print(len(graph.edges))
    
    #print(len(graph_0001.edges))
    #pickle.dump(graph, open(str(re.sub('_seg.npy','.pickle',file)), 'w'))
    #nx.write_pajek(graph,re.sub('_seg.npy','.pajek',file))
    nx.write_gpickle(graph,re.sub('_seg_warped.tif','_warped.pickle',file))
    #raph.to_csv(re.sub('seg.npy','graph.csv',file))

  0%|          | 0/94 [00:00<?, ?it/s]

james_preds/XYZres006_skel_warped_single.mat


/tmp/ipykernel_384624/1148313062.py:16: UserWarning: james_preds/XYZres006_0001_dst_skel_warped_single.tif is a low contrast image
  io.imsave(re.sub('_seg_warped_filled.tif','_dst_skel_warped_single.tif',file),skel_dst)
  1%|          | 1/94 [00:16<25:45, 16.62s/it]

james_preds/XYZres006_skel_warped_single.mat


/tmp/ipykernel_384624/1148313062.py:16: UserWarning: james_preds/XYZres006_dst_skel_warped_single.tif is a low contrast image
  io.imsave(re.sub('_seg_warped_filled.tif','_dst_skel_warped_single.tif',file),skel_dst)
  2%|▏         | 2/94 [00:31<23:58, 15.64s/it]

...

james_preds/XYZres007_skel_warped_single.mat


/tmp/ipykernel_384624/1148313062.py:16: UserWarning: james_preds/XYZres007_dst_skel_warped_single.tif is a low contrast image
  io.imsave(re.sub('_seg_warped_filled.tif','_dst_skel_warped_single.tif',file),skel_dst)
  4%|▍         | 4/94 [01:06<25:07, 16.75s/it]

james_preds/XYZres008_skel_warped_single.mat


  4%|▍         | 4/94 [01:19<29:42, 19.81s/it]


KeyboardInterrupt: 

In [12]:
graph[0]

AtlasView({112: {'pts': array([[  3,  68, 153],
       [  4,  68, 153],
       [  5,  68, 153],
       [  6,  68, 153],
       [  7,  68, 153],
       [  8,  68, 153],
       [  9,  69, 152],
       [ 10,  69, 152],
       [ 11,  69, 152],
       [ 12,  69, 152],
       [ 13,  70, 152],
       [ 14,  70, 152],
       [ 15,  70, 152],
       [ 16,  70, 152]], dtype=int16), 'weight': 14.146264369941973}})

In [ ]:
re.sub('_seg.npy','.pickle',file)

In [ ]:
skel_file = re.sub('seg.npy','skel.mat',file)

In [ ]:
graph[0][39]

In [ ]:
re.sub('seg.npy','graph.csv',file_0001)

In [ ]:
files[22:24]